In [1]:
import parmed as prm
import pytraj as pt
import sys
import collections
import pandas as pd
import numpy as np
import nglview as nv
import matplotlib
from matplotlib import pyplot as plt
import gc
import copy

In [2]:
parmName='continuousPh/structures/WT.parm7'
rstName='continuousPh/structures/WT.rst7'
parmDat=prm.load_file(parmName)
parmDat.load_rst7(rstName)
parmDat

<AmberParm 6887 atoms; 435 residues; 6962 bonds; parametrized>

In [3]:
#this will get put into an external file in the near future.
phmdparmDataDict={
    'AS2':{
        'NUMCH':14,
        'RES_NAME':'AS2',
        'RES_TYPE':4,
        'ATOM_NAME':[ 'N','H','CA','HA','CB','HB2','HB3','CG','OD1','OD2',
                     'HD2','C','O','HD1'],
        'CH':[ -0.415700,0.271900,0.034100,0.086400,-0.031600,0.048800,
              0.048800,0.646200,-0.637600,-0.555400,0.00,0.597300,-0.567900,
              0.4747,-0.415700,0.271900,0.034100,0.086400,-0.031600,0.048800,
              0.048800,0.646200,-0.555400,-0.637600,0.474700,0.597300,-0.567900,
              0.00,-0.415700,0.271900,0.034100,0.086400,-0.178200,-0.012200,
              -0.012200,0.799400,-0.801400,-0.801400,0.00,0.597300,-0.567900,0.00],
        'CH_MD':[-0.415700,0.271900,0.034100,0.086400,-0.031600,0.048800,
                 0.048800,0.646200,-0.637600,-0.555400,0.00,0.597300,-0.567900,0.4747,
                 -0.415700,0.271900,0.034100,0.086400,-0.031600,0.048800,0.048800,0.646200,
                 -0.555400,-0.637600,0.474700,0.597300,-0.567900,0.00,-0.516300,0.293600,
                 0.038100,0.088000,-0.030300,-0.012200,-0.012200,0.799400,-0.801400,
                 -0.801400,0.00,0.536600,-0.581900,0.00],
        'RAD':[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,
               0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,
               0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,],
        'MODEL_PKA':[3.5],
        'BAR':[2.5,2.5],
        'PARAMETERS':[-60.0334,0.305108,-59.876,0.304068,-21.555,
                      0.497647,-19.9303,41.6005,-21.5678,0.51803,-59.8132,0.304052]
    },
    'GL2':{
        'NUMCH':17,
        'RES_NAME':'GL2',
        'RES_TYPE':4,
        'ATOM_NAME':['N','H','CA','HA','CB','HB2','HB3','CG','HG2','HG3',
                     'CD','OE1','OE2','HE2','C','O','HE1'],
        'CH':[-0.415700,0.271900,0.014500,0.077900,-0.007100,0.025600,
              0.025600,-0.017400,0.043000,0.043000,0.680100,-0.651100,-0.583800,
              0.00,0.597300,-0.567900,0.4641,-0.415700,0.271900,0.014500,
              0.077900,-0.007100,0.025600,0.025600,-0.017400,0.043000,0.043000,
              0.680100,-0.583800,-0.651100,0.464100,0.597300,-0.567900,0.00,
              -0.415700,0.271900,0.014500,0.077900,-0.039800,-0.017300,-0.017300,
              0.013600,-0.042500,-0.042500,0.805400,-0.818800,-0.818800,0.00,0.597300,
              -0.567900,0.00],
        'CH_MD':[ -0.415700,0.271900,0.014500,0.077900,-0.007100,0.025600,
                 0.025600,-0.017400,0.043000,0.043000,0.680100,-0.651100,-0.583800,
                 0.00,0.597300,-0.567900,0.4641,-0.415700,0.271900,0.014500,0.077900,
                 -0.007100,0.025600,0.025600,-0.017400,0.043000,0.043000,0.680100,
                 -0.583800,-0.651100,0.464100,0.597300,-0.567900,0.00,-0.516300,
                 0.293600,0.039700,0.110500,0.056000,-0.017300,-0.017300,0.013600,
                 -0.042500,-0.042500,0.805400,-0.818800,-0.818800,0.00,0.536600,-0.581900,
                 0.00],
        'RAD':[ 0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
               0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
               1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
               0.0,0.0,0.0,0.0,0.0],
        'MODEL_PKA':[4.2],
        'BAR':[2.5,2.5],
        'PARAMETERS':[-52.4396,0.447635,-52.1586,0.447125,-23.1202,
                      0.499933,-21.5247,44.8027,-23.166,0.50406,-52.1457,0.447245]},
    'HIP':{
        'NUMCH':18,
        'RES_NAME':'HIP',
        'RES_TYPE':2,
        'ATOM_NAME':['N','H','CA','HA','CB','HB2','HB3','CG','ND1','HD1',
                     'CE1','HE1','NE2','HE2','CD2','HD2','C','O'],
        'CH':[ -0.347900,0.274700,-0.135400,0.121200,-0.041400,0.081000,
              0.081000,-0.001200,-0.151300,0.386600,-0.017000,0.268100,
              -0.171800,0.391100,-0.114100,0.231700,0.734100,-0.589400,-0.347900,
              0.274700,-0.135400,0.121200,-0.111000,0.040200,0.040200,-0.026600,
              -0.381100,0.364900,0.205700,0.139200,-0.572700,0.00,0.129200,
              0.114700,0.734100,-0.589400,-0.347900,0.274700,-0.135400,0.121200,
              -0.101200,0.036700,0.036700,0.18680,-0.543200,0.00,0.163500,0.143500,
              -0.279500,0.333900,-0.220700,0.186200,0.734100,-0.589400],
        'CH_MD':[-0.347900,0.274700,-0.135400,0.121200,-0.041400,0.081000,
                 0.081000,-0.001200,-0.151300,0.386600,-0.017000,0.268100,-0.171800,
                 0.391100,-0.114100,0.231700,0.734100,-0.589400,-0.415700,0.271900,
                 0.018800,0.088100,-0.046200,0.040200,0.040200,-0.026600,-0.381100,
                 0.364900,0.205700,0.139200,-0.572700,0.00,0.129200,0.114700,0.597300,
                 -0.567900,-0.415700,0.271900,-0.058100,0.136000,-0.007400,0.036700,
                 0.036700,0.18680,-0.543200,0.00,0.163500,0.143500,-0.279500,0.333900,
                 -0.220700,0.186200,0.597300,-0.567900],
        'RAD':[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,
               0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
               0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
               0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0],
        'MODEL_PKA':[6.1,6.6],
        'BAR':[2.5,2.5],
        'PARAMETERS':[-47.5760265552,0.487738121,-45.3526708755]},
    'LYS':{
        'NUMCH':22,
        'RES_NAME':'LYS',
        'RES_TYPE':0,
        'ATOM_NAME':['N','H','CA','HA','CB','HB2','HB3','CG','HG2','HG3',
                     'CD','HD2','HD3','CE','HE2','HE3','NZ','HZ1','HZ2','HZ3','C','O'],
        'CH':[-0.347900,0.274700,-0.240000,0.142600,-0.009400,0.036200,
              0.036200,0.018700,0.010300,0.010300,-0.047900,0.062100,0.062100,
              -0.014300,0.113500,0.113500,-0.38540,0.340000,0.340000,0.340000,
              0.734100,-0.589400,-0.347900,0.274700,-0.240000,0.142600,-0.109600,
              0.034000,0.034000,0.066120,0.010410,0.010410,-0.037680,0.011550,
              0.011550,0.326040,-0.033580,-0.033580,-1.035810,0.00,0.386040,
              0.386040,0.734100,-0.589400],
        'CH_MD':[-0.347900,0.274700,-0.240000,0.142600,-0.009400,0.036200,
                 0.036200,0.018700,0.010300,0.010300,-0.047900,0.062100,0.062100,
                 -0.014300,0.113500,0.113500,-0.38540,0.340000,0.340000,0.340000,
                 0.734100,-0.589400,-0.415700,0.271900,-0.072060,0.099400,-0.048450,
                 0.034000,0.034000,0.066120,0.010410,0.010410,-0.037680,0.011550,
                 0.011550,0.326040,-0.033580,-0.033580,-1.035810,0.00,0.386040,
                 0.386040,0.597300,-0.567900,],
        'RAD':[ 0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
               0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
               0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],
        'MODEL_PKA':[10.4],
        'BAR':[2.5],
        'PARAMETERS':[-55.665,0.6655]},
    'CYS':{
        'NUMCH':11,
        'RES_NAME':'CYS',
        'RES_TYPE':0,
        'ATOM_NAME':['N','H','CA','HA','CB','HB2','HB3','SG','HG','C','O'],
        'CH':[-0.415700,0.271900,0.021300,0.112400,-0.123100,0.111200,
              0.111200,-0.311900,0.193300,0.597300,-0.567900,-0.415700,0.271900,
              0.021300,0.112400,-0.3593,0.112200,0.112200,-0.884400,0.00,0.597300,
              -0.567900],
        'CH_MD':[-0.415700,0.271900,0.021300,0.112400,-0.123100,0.111200,
                 0.111200,-0.311900,0.193300,0.597300,-0.567900,-0.415700,0.271900,
                 -0.035100,0.050800,-0.241300,0.112200,0.112200,-0.884400,0.00,0.597300,
                 -0.567900],
        'RAD':[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,
               0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],
        'MODEL_PKA':[8.5],
        'BAR':[2.5],
        'PARAMETERS':[ -80.3161983,0.053018419]}
}

In [4]:
def get_ResnumList(resnameList,parmedData,phmdDict):
    resnames=np.unique(resnameList)
    return list(np.sort([res.idx for res in parmedData.residues if res.name in resnames]))

def get_Resnames(resnumList,parmedData):
    return map(lambda x: parmedData.residues[x].name,resnumList)

def check_ResnumList(resnumList,parmedData,phmdDict):
    resNames=get_Resnames(resnumList,parmedData)
    checkList=map(lambda x: x in phmdDict,resNames)
    return checkList


In [5]:
def make_phmdparm_lines(resnumList,parmedData,phmdDict):
    #will need to implement a sanity check in the future
    #to make sure we don't try to query residues that
    #do not have a listing in phmdDict dictionary
    #validate_resnumList(resnumList,parmedData,phmdDict)
    
    #get residue names by querying parmedData for residues
    #at the indices listed in resnumList
    #These will be automatically sorted to ensure ascending order
    resnames=get_Resnames(
        list(np.sort(resnumList)),parmedData)
    
    #start building the lines for the &phmdparm namelist
    #'NGT' is just a count of the number of residues
    phmdparmLines=['   NGT = %g,'%len(resnames)]
    
    #'NUMCH(:)' lists how many atoms are in each residue
    phmdparmLines.append(
        '   NUMCH(:) = '+ \
        ','.join([
            '%s'%phmdDict[resname]['NUMCH'] \
            for resname in resnames
        ]) + ',')
    
    #'RES_NAME(:)'' lists the names of each residue
    phmdparmLines.append(
        '   RES_NAME(:) = ' + \
        ','.join([
            "'%s'"%phmdDict[resname]['RES_NAME'] \
            for resname in resnames
        ]) + ',')
    
    #'RES_TYPE(:)' lists the titratable residue type for each residue
    #this is either 4, 2, or 0 (see continuous ph section of AMBER manual)
    phmdparmLines.append(
        '   RES_TYPE(:) = '+ \
        ','.join([
            '%g'%phmdDict[resname]['RES_TYPE'] \
            for resname in resnames
        ]))
    
    #'ATOM_NAME(ii, :)' make a separate array entry for each residue
    #which contains the name of each atom in that residue
    for iRes,res in enumerate(resnames):
        ii=iRes+1
        atomNames=phmdDict[res]['ATOM_NAME']
        phmdparmLines.append(
            '   ATOM_NAME(%g, :) = '%(ii) + \
            ','.join([
                "'%s'"%atmName for atmName in atomNames
            ]) + ',')
    
    #'CH(ii, :)' make a separate array entry for each residue
    #which contains the charge for each atom under each protonation
    #state w.r.t. lambda coordinates
    for iRes,res in enumerate(resnames):
        ii=iRes+1
        CH_vals=phmdDict[res]['CH']
        phmdparmLines.append(
            '   CH(%g, :) = '%(ii) + \
            ','.join([
                "%f"%CH_val for CH_val in CH_vals
            ]) + ',')
    
    #'CH_MD(ii, :)' make a separate array entry for each residue
    #which contains the charge for each atom under each protonation
    #state w.r.t. spatial coordinates
    for iRes,res in enumerate(resnames):
        ii=iRes+1
        CH_MD_vals=phmdDict[res]['CH_MD']
        phmdparmLines.append(
            '   CH_MD(%g, :) = '%(ii) + \
            ','.join([
                "%f"%CH_MD_val for CH_MD_val in CH_MD_vals
            ]) + ',')
    
    #'RAD(ii, :)' make a separate array entry for each residue
    #which contains a set of masks for each atom in each state
    #that tell phmd what atoms are appearing / disappearing
    #see constant ph section of AMBER manual for more details
    for iRes,res in enumerate(resnames):
        ii=iRes+1
        RAD_vals=phmdDict[res]['RAD']
        phmdparmLines.append(
            '   RAD(%g, :) = '%(ii) + \
            ','.join([
                "%.1f"%RAD_val for RAD_val in RAD_vals
            ]) + ',')
    
    #'MODEL_PKA(ii, :)' make a separate array entry for each residue
    #which contains the pKa of each protonation site type
    #at the moment, only HIP has more than one. For GL2 and AS2
    #the two protonation sites are equivalent.
    for iRes,res in enumerate(resnames):
        ii=iRes+1
        MODEL_PKA_vals=phmdDict[res]['MODEL_PKA']
        phmdparmLines.append(
            '   MODEL_PKA(%g, :) = '%(ii) + \
            ','.join([
                "%.3f"%MODEL_PKA_val for MODEL_PKA_val in MODEL_PKA_vals
            ]) + ',')
    
    #'BAR(ii, :)' make a separate array entry for each residue
    #which contains the barrier heights for lambda and, if necessary,
    #spatial transitions
    for iRes,res in enumerate(resnames):
        ii=iRes+1
        BAR_vals=phmdDict[res]['BAR']
        phmdparmLines.append(
            '   BAR(%g, :) = '%(ii) + \
            ','.join([
                "%.3f"%BAR_val for BAR_val in BAR_vals
            ]) + ',')
    
    #'PARAMETERS(ii, :)' make a separate array entry for each residue
    #which contains relevant potential model parameters
    #see continuous ph section of AMBER manual for more details
    for iRes,res in enumerate(resnames):
        ii=iRes+1
        PARAMETERS_vals=phmdDict[res]['PARAMETERS']
        phmdparmLines.append(
            '   PARAMETERS(%g, :) = '%(ii) + \
            ','.join([
                "%.3f"%PARAMETERS_val for PARAMETERS_val in PARAMETERS_vals
            ]) + ',')
        
    return phmdparmLines

def print_phmdparmLines_to_file(filepath,phmdparmLines,
                                header='phmdparm file'):
    outFile=open(filepath,'w')
    outFile.write(header+'\n')
    outFile.write('&phmdparm'+'\n')
    for line in phmdparmLines:
        outFile.write(line)
        outFile.write('\n')
    outFile.write('/')
    outFile.close()

In [6]:
resnums=list(np.sort(get_ResnumList(['AS2','HIP','LYS','CYS'],parmDat,phmdparmDataDict)))
print '\n'.join(make_phmdparm_lines(resnums,parmDat,phmdparmDataDict))
print_phmdparmLines_to_file('continuousPh/WT.phmdparm',
                            make_phmdparm_lines(resnums,parmDat,phmdparmDataDict),
                            header='phmdparm file')

   NGT = 47,
   NUMCH(:) = 18,11,11,22,22,22,18,18,18,18,11,18,18,22,18,22,22,22,22,11,11,14,18,22,11,22,22,22,11,22,22,11,22,22,22,22,11,18,22,22,22,22,22,22,18,18,22,
   RES_NAME(:) = 'HIP','CYS','CYS','LYS','LYS','LYS','HIP','HIP','HIP','HIP','CYS','HIP','HIP','LYS','HIP','LYS','LYS','LYS','LYS','CYS','CYS','AS2','HIP','LYS','CYS','LYS','LYS','LYS','CYS','LYS','LYS','CYS','LYS','LYS','LYS','LYS','CYS','HIP','LYS','LYS','LYS','LYS','LYS','LYS','HIP','HIP','LYS',
   RES_TYPE(:) = 2,0,0,0,0,0,2,2,2,2,0,2,2,0,2,0,0,0,0,0,0,4,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,2,2,0
   ATOM_NAME(1, :) = 'N','H','CA','HA','CB','HB2','HB3','CG','ND1','HD1','CE1','HE1','NE2','HE2','CD2','HD2','C','O',
   ATOM_NAME(2, :) = 'N','H','CA','HA','CB','HB2','HB3','SG','HG','C','O',
   ATOM_NAME(3, :) = 'N','H','CA','HA','CB','HB2','HB3','SG','HG','C','O',
   ATOM_NAME(4, :) = 'N','H','CA','HA','CB','HB2','HB3','CG','HG2','HG3','CD','HD2','HD3','CE','HE2','HE3','NZ','HZ1','HZ2','HZ3','C','O',
   ATOM_NAME(